# Support Vector Machine Learning Model (94% variance - F1 Macro)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (94% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.94)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,3256,3257,3258,3259,3260,3261,3262,3263,3264,3265
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.009400,0.006561,-0.009712,-0.011965,0.002525,-0.013138,0.022994,-0.003856,-0.004976,0.013235
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.000490,-0.006393,-0.003049,0.003465,0.002297,0.005555,0.001494,-0.004797,-0.001776,-0.002160
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.002074,-0.000058,-0.009895,-0.013750,-0.005589,-0.002226,0.001246,-0.004731,-0.009128,0.003801
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,0.015568,-0.003044,0.007082,0.005148,-0.014525,0.005504,0.005235,0.007880,-0.014091,-0.010293
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.003672,-0.010175,-0.006867,-0.007640,-0.008024,0.003959,-0.008504,0.000014,-0.001714,0.005093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.001457,-0.001991,0.004962,-0.001259,-0.003267,-0.006454,0.002295,-0.001124,0.008736,-0.009022
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.007384,-0.010612,0.007094,-0.008741,0.002143,0.012900,0.013886,0.001384,-0.003357,0.003352
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.007339,-0.006330,-0.007820,-0.008767,0.014970,-0.000341,-0.011852,0.001511,0.005921,0.008343
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.004688,-0.000729,-0.000392,0.003280,-0.000686,0.001717,-0.000905,0.001238,-0.000993,-0.000830


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,3256,3257,3258,3259,3260,3261,3262,3263,3264,3265
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.009400,0.006561,-0.009712,-0.011965,0.002525,-0.013138,0.022994,-0.003856,-0.004976,0.013235
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.000490,-0.006393,-0.003049,0.003465,0.002297,0.005555,0.001494,-0.004797,-0.001776,-0.002160
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,0.002074,-0.000058,-0.009895,-0.013750,-0.005589,-0.002226,0.001246,-0.004731,-0.009128,0.003801
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,0.015568,-0.003044,0.007082,0.005148,-0.014525,0.005504,0.005235,0.007880,-0.014091,-0.010293
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.003672,-0.010175,-0.006867,-0.007640,-0.008024,0.003959,-0.008504,0.000014,-0.001714,0.005093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.001457,-0.001991,0.004962,-0.001259,-0.003267,-0.006454,0.002295,-0.001124,0.008736,-0.009022
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.007384,-0.010612,0.007094,-0.008741,0.002143,0.012900,0.013886,0.001384,-0.003357,0.003352
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.007339,-0.006330,-0.007820,-0.008767,0.014970,-0.000341,-0.011852,0.001511,0.005921,0.008343
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.004688,-0.000729,-0.000392,0.003280,-0.000686,0.001717,-0.000905,0.001238,-0.000993,-0.000830


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (94% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,3256,3257,3258,3259,3260,3261,3262,3263,3264,3265
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,0.001727,-0.001643,-0.003937,0.000552,0.004349,0.002439,-0.001332,0.002036,-0.003411,-0.001329
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,0.002377,0.005854,-0.002690,0.008686,0.013980,-0.005390,0.012853,-0.010026,0.007168,-0.007079
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,0.010351,-0.003277,0.001219,0.008931,0.004128,0.001111,0.011600,-0.005195,-0.002951,-0.007670
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,-0.003603,-0.006894,0.003594,0.003951,-0.007252,0.006969,0.004807,-0.005723,-0.006318,0.003148
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,-0.000773,0.001469,0.000256,0.002723,0.003993,0.000309,0.000310,-0.002378,-0.000874,0.000202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,-0.001100,-0.002793,0.000338,-0.000480,0.000942,-0.003520,0.002443,-0.007677,-0.000640,-0.002460
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,0.007413,0.003558,0.000603,-0.002042,-0.009680,-0.001738,-0.010859,-0.003052,-0.000633,-0.005981
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,-0.004294,0.012151,0.012872,-0.007891,-0.019562,0.022001,0.019556,-0.007449,-0.012113,-0.002730
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,-0.001511,0.000175,-0.006101,0.003567,-0.004666,0.011748,0.006014,0.000013,-0.004582,0.006094


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 6.1min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 6.3min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 6.4min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.8min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.9min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.8min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.4min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.2min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.1min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.4min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [10]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_94_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_94_f1macro.csv